# 50.040 Natural Language Processing (Fall 2025) Homework 3

**Due 30/11/2025, 23:59 PM**


### STUDNET ID: 1006922

### Name: Lim Sherri

### Students with whom you have discussed (if any):

## IMDB Sentiment Analysis

### Project Overview

This notebook implements a complete sentiment analysis pipeline for IMDB movie reviews using:
- **Model**: Qwen/Qwen3-0.6B (600M parameters)
- **Dataset**: stanfordnlp/imdb (50,000 movie reviews)
- **Task**: Binary sentiment classification (positive/negative)
- **Course**: 50.040 / NLP — IMDB Sentiment homework 3

### Methodology

We compare three approaches:
1. **Base Model**: Zero-shot performance without fine-tuning
2. **Full-Parameter SFT**: Fine-tuning all model parameters
3. **LoRA-SFT**: Parameter-efficient fine-tuning using LoRA (Low-Rank Adaptation)

Tips: If you don't have a GPU, we recommend using Kaggle & Colab resources to complete this experiment. Estimated GPU processing time is <2 hours.

Kaggle:  https://www.kaggle.com/

Colab: https://colab.research.google.com/


PS: It is important that all code runs completely, accounting for 10 points.

---
## Section 0: Environment Setup and Configuration

This section imports all necessary libraries, sets random seeds for reproducibility, and configures the training environment.

In [3]:
!pip install jsonlines

  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
Using cached jsonlines-4.0.0-py3-none-any.whl (8.7 kB)


In [4]:
"""
Import required libraries and set environment variables
"""
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import json
import random
import numpy as np
import torch
import jsonlines
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    set_seed
)
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib
matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
from typing import Dict, List, Tuple
import gc
from torch.utils.data import Dataset

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [5]:
"""
Set random seed for reproducibility across all frameworks
"""
RANDOM_SEED = 42
set_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print(f"Random seed set to: {RANDOM_SEED}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

Random seed set to: 42
PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA device: Tesla T4
CUDA memory: 14.74 GB


In [6]:
"""
GPU memory management utility

This function helps free up GPU memory between training runs
and provides visibility into current memory usage.
"""
def free_gpu(tag: str = ""):
    """Clean up GPU memory and print current usage."""
    gc.collect()
    if not torch.cuda.is_available():
        print(f"[free_gpu]{'['+tag+']' if tag else ''} CUDA not available.")
        return

    for d in range(torch.cuda.device_count()):
        try:
            torch.cuda.synchronize(d)
        except:
            pass
        with torch.cuda.device(d):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
        try:
            torch.cuda.synchronize(d)
        except:
            pass

    gc.collect()

    # Print memory usage
    stats = []
    for d in range(torch.cuda.device_count()):
        a = torch.cuda.memory_allocated(d) / 1048576
        r = torch.cuda.memory_reserved(d) / 1048576
        stats.append(f"GPU{d}: alloc={a:.1f}MB, reserved={r:.1f}MB")
    print(f"[free_gpu]{'['+tag+']' if tag else ''} " + " | ".join(stats))

free_gpu("initial")

[free_gpu][initial] GPU0: alloc=0.0MB, reserved=0.0MB


In [7]:
"""
Configuration - Optimized for 15GB GPU

These hyperparameters are carefully tuned to:
- Fit within 15GB GPU memory constraints
- Provide reasonable training time (~2-5 minutes per approach)
- Maintain model performance
"""
MODEL_NAME = "Qwen/Qwen3-0.6B"
MAX_SEQ_LENGTH = 512
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_EPOCHS = 1
WARMUP_STEPS = 20
MAX_TRAIN_SAMPLES = 2000
MAX_VALID_SAMPLES = 100
MAX_TEST_SAMPLES = 100
GRADIENT_ACCUMULATION_STEPS = 4


# Create directory structure
os.makedirs("data", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("figures", exist_ok=True)
os.makedirs("checkpoints/full_sft", exist_ok=True)
os.makedirs("checkpoints/lora_sft", exist_ok=True)

print("\n" + "="*60)
print("Configuration:")
print("="*60)
print(f"Model: {MODEL_NAME}")
print(f"Max sequence length: {MAX_SEQ_LENGTH}")
print(f"Train batch size: {TRAIN_BATCH_SIZE}")
print(f"Eval batch size: {EVAL_BATCH_SIZE}")
print(f"Gradient accumulation steps: {GRADIENT_ACCUMULATION_STEPS}")
print(f"Effective batch size: {TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Number of epochs: {NUM_EPOCHS}")
print(f"Train samples: {MAX_TRAIN_SAMPLES}")
print(f"Validation samples: {MAX_VALID_SAMPLES}")
print(f"Test samples: {MAX_TEST_SAMPLES}")
print("="*60 + "\n")


Configuration:
Model: Qwen/Qwen3-0.6B
Max sequence length: 512
Train batch size: 8
Eval batch size: 8
Gradient accumulation steps: 4
Effective batch size: 32
Learning rate: 2e-05
Number of epochs: 1
Train samples: 2000
Validation samples: 100
Test samples: 100



---
## Section 1: Data Processing

In this section, we:
1. Load the IMDB dataset from HuggingFace
2. Convert reviews to instruction-following format
3. Split data into train, validation, and test sets
4. Save processed data in JSONL format

In [8]:
"""
Define data formatting functions

The instruction format helps the model understand the task:
Prompt: "Is this review positive or negative?\n\n{review}\n\nAnswer with only one word: positive or negative. Answer:"
Response: "positive" or "negative"
"""
def format_instruction(text: str, label: int) -> Dict[str, str]:
    """
    Convert IMDB sample to instruction format.

    Args:
        text: Original review text
        label: 0 (negative) or 1 (positive)

    Returns:
        Dictionary with prompt and response
    """
    #
    sentiment = "positive" if label == 1 else "negative"
    return {
        "prompt": f"Is this review positive or negative?\n\n{text}\n\nAnswer with only one word: positive or negative. Answer:",
        "response": sentiment,
        "label": label
    }

def save_dataset(data, output_path: str):
    """Save dataset to JSONL format."""
    with jsonlines.open(output_path, mode='w') as writer:
        for item in data:
            formatted = format_instruction(item['text'], item['label'])
            writer.write(formatted)
    print(f"Saved {len(data)} samples to {output_path}")

print("✅ Data formatting functions defined")

✅ Data formatting functions defined




#### Q1.1: Why do we need to reformat the dataset instead of using the original one directly? Explain the purpose of converting the raw IMDB reviews into an instruction-following format. [written] (4 point)





**ANSWER**: The LLM requires instruction-response pairs, but the raw IMDB reviews only contain reviews and their respective sentiment labels. By converting  them to an instruction-following format, we help the model learn how to interpret and follow user instructions in an effective and predictable way.


#### Q1.2: How can we train LLM if we don’t perform this conversion? Discuss the implications of using raw data versus formatted instruction data for language model training. [written] (4 point)




**ANSWER**: The LLM can be trained with a traditional sequence classification approach, taking in the input text and predicting a sentiment label directly (e.g. using a feed forward neural network). However, wtith raw data alone, the model will be unable to understand and execute natural language instructions.

#### Q1.3: Complete the data splitting and sampling implementation in the notebook:

*   Split the training data into train and validation sets
*   Perform random sampling to limit dataset size
*   Ensure balanced class distribution

In [9]:
"""
Load and split IMDB dataset

The IMDB dataset contains 50,000 movie reviews:
- 25,000 for training
- 25,000 for testing

We subsample to reduce training time while maintaining balanced classes.
"""
print("Loading IMDB dataset...")
dataset = load_dataset("stanfordnlp/imdb")

print(f"Original train samples: {len(dataset['train'])}")
print(f"Original test samples: {len(dataset['test'])}")

# Split train into train and validation    (7 point)
# Sample training data & random selection
### YOUR CODE HERE
random.seed(RANDOM_SEED)
train_data_full = dataset['train']

train_pos_indices = [i for i, item in enumerate(train_data_full) if item['label']==1]
train_neg_indices = [i for i, item in enumerate(train_data_full) if item['label']==0]
train_pos_sample = random.sample(train_pos_indices, MAX_TRAIN_SAMPLES // 2)
train_neg_sample = random.sample(train_neg_indices, MAX_TRAIN_SAMPLES // 2)
train_subset_indices = train_pos_sample + train_neg_sample
random.shuffle(train_subset_indices)

train_subset = []
for idx in train_subset_indices:
    item = train_data_full[idx]
    train_subset.append({
        'text': item['text'],
        'label': 'positive' if item['label'] == 1 else 'negative'
    })
# Now do for the validation samples
train_pos_rem = list(set(train_pos_indices)-set(train_pos_sample))
train_neg_rem = list(set(train_neg_indices)-set(train_neg_sample))

val_pos_sample = random.sample(train_pos_rem, MAX_VALID_SAMPLES // 2)
val_neg_sample = random.sample(train_neg_rem, MAX_VALID_SAMPLES // 2)
valid_subset_indices = val_pos_sample + val_neg_sample
random.shuffle(valid_subset_indices)

valid_subset = []
for idx in valid_subset_indices:
    item = train_data_full[idx]
    valid_subset.append({
        'text': item['text'],
        'label': 'positive' if item['label'] == 1 else 'negative'
    })
### END OF YOUR CODE

# Limit test data
### YOUR CODE HERE
test_data_full = dataset['test']
test_pos_indices = [i for i, item in enumerate(test_data_full) if item['label']==1]
test_neg_indices = [i for i, item in enumerate(test_data_full) if item['label']==0]
test_pos_sample = random.sample(test_pos_indices, MAX_TEST_SAMPLES // 2)
test_neg_sample = random.sample(test_neg_indices, MAX_TEST_SAMPLES // 2)
test_subset_indices = test_pos_sample + test_neg_sample
random.shuffle(test_subset_indices)

test_subset = []
for idx in test_subset_indices:
    item = test_data_full[idx]
    test_subset.append({
        'text': item['text'],
        'label': 'positive' if item['label'] == 1 else 'negative'
    })
### END OF YOUR CODE

print(f"\nDataset splits:")
print(f"Train: {len(train_subset)} samples")
print(f"Validation: {len(valid_subset)} samples")
print(f"Test: {len(test_subset)} samples")

# Save to JSONL
save_dataset(train_subset, "data/train.jsonl")
save_dataset(valid_subset, "data/valid.jsonl")
save_dataset(test_subset, "data/test.jsonl")

print("\n✅ Data processing complete")

Loading IMDB dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Original train samples: 25000
Original test samples: 25000

Dataset splits:
Train: 2000 samples
Validation: 100 samples
Test: 100 samples
Saved 2000 samples to data/train.jsonl
Saved 100 samples to data/valid.jsonl
Saved 100 samples to data/test.jsonl

✅ Data processing complete


---
## Section 2: Tokenizer Setup

Load the tokenizer for the Qwen model and configure special tokens for padding.

In [10]:
"""
Load and configure tokenizer

The tokenizer converts text to token IDs that the model understands.
We set the pad_token to eos_token if not already defined.
"""
print(f"Loading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# Set pad token if not exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Set pad_token to eos_token")

print(f"\nTokenizer vocab size: {len(tokenizer)}")
print(f"PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")
print("\n✅ Tokenizer loaded successfully")

Loading tokenizer from Qwen/Qwen3-0.6B...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]


Tokenizer vocab size: 151669
PAD token: <|endoftext|> (ID: 151643)
EOS token: <|im_end|> (ID: 151645)

✅ Tokenizer loaded successfully


---
## Section 3: Dataset Class

Define a custom PyTorch Dataset class for instruction-following format with:
- Proper tokenization of prompts and responses
- Label masking (only train on response tokens, not prompt)
- Padding and truncation to max length

In [11]:
"""
Helper function to load JSONL files
"""
def load_jsonl(file_path: str) -> List[Dict]:
    """Load JSONL file."""
    data = []
    with jsonlines.open(file_path) as reader:
        for obj in reader:
            data.append(obj)
    return data

print("✅ JSONL loader defined")

✅ JSONL loader defined


In [12]:
"""
Define custom Dataset class for instruction-following

Key features:
1. Tokenizes prompt and response separately
2. Sets labels to -100 for prompt tokens (ignored in loss)
3. Sets labels to actual tokens for response (trained)
4. Handles padding and truncation
"""
class InstructionDataset(Dataset):
    def __init__(self, file_path: str, tokenizer, max_length: int):
        self.data = load_jsonl(file_path)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        prompt = item['prompt']
        response = item['response']

        # Tokenize prompt and response separately
        prompt_encoded = self.tokenizer(
            prompt,
            add_special_tokens=True,
            truncation=False
        )
        response_text = f" {response}{self.tokenizer.eos_token}"
        response_encoded = self.tokenizer(
            response_text,
            add_special_tokens=False,
            truncation=False
        )

        # Combine input_ids  & output (2 point)
        ### YOUR CODE HERE
        input_ids = prompt_encoded['input_ids'] + response_encoded['input_ids']
        attention_mask=[1]*len(input_ids) # Model attend to all real tokens
        # Create labels: -100 for prompt (ignored), actual tokens for response (2 point)
        ### YOUR CODE HERE
        labels = [-100]*len(prompt_encoded['input_ids'])+response_encoded['input_ids']

        # Truncate (3 point)
        ### YOUR CODE HERE
        if len(input_ids) > self.max_length:
            input_ids = input_ids[:self.max_length]
            attention_mask = attention_mask[:self.max_length]
            labels = labels[:self.max_length]

        # Pad to max_length (3 point)
        pad_length = self.max_length - len(input_ids)
        if pad_length > 0:
          input_ids += [self.tokenizer.pad_token_id] * pad_length
          attention_mask += [0] * pad_length # model will ignore padding tokens
          labels += [-100] * pad_length # loss is ignored for padding tokens
        ### YOUR CODE HERE

        return {
            'input_ids': torch.tensor(input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'labels': torch.tensor(labels, dtype=torch.long)
        }

print("✅ InstructionDataset class defined")

✅ InstructionDataset class defined


In [13]:
"""
Create dataset instances for train, validation, and test
"""
train_dataset = InstructionDataset("data/train.jsonl", tokenizer, MAX_SEQ_LENGTH)
valid_dataset = InstructionDataset("data/valid.jsonl", tokenizer, MAX_SEQ_LENGTH)
test_dataset = InstructionDataset("data/test.jsonl", tokenizer, MAX_SEQ_LENGTH)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Valid dataset size: {len(valid_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# Verify dataset
print("\n" + "="*60)
print("TOKENIZED DATASET SAMPLE INSPECTION")
print("="*60)
sample = train_dataset[0]
print(f"Dataset sample shapes:")
print(f"  Input IDs shape: {sample['input_ids'].shape}")
print(f"  Labels shape: {sample['labels'].shape}")
print(f"  Attention mask shape: {sample['attention_mask'].shape}")

print(f"\nInput IDs (first 20 tokens): {sample['input_ids'][:20].tolist()}")
print(f"Labels (first 20 tokens): {sample['labels'][:20].tolist()}")
print(f"Attention mask (first 20 tokens): {sample['attention_mask'][:20].tolist()}")

# Decode to verify
decoded_input = tokenizer.decode(sample['input_ids'], skip_special_tokens=False)
print(f"\nDecoded input (first 200 chars): {decoded_input[:200]}...")

# Show where labels start being actual tokens (not -100)
label_start = None
for i, label in enumerate(sample['labels']):
    if label != -100:
        label_start = i
        break
print(f"\nLabels analysis:")
print(f"  Prompt tokens (label=-100): {label_start}")
print(f"  Response tokens (label=actual): {(sample['labels'] != -100).sum().item()}")
print(f"  Padding tokens (label=-100): {((sample['labels'] == -100) & (sample['attention_mask'] == 0)).sum().item()}")

# Decode only the response part
if label_start is not None:
    response_tokens = sample['input_ids'][label_start:]
    response_tokens = response_tokens[response_tokens != tokenizer.pad_token_id]
    decoded_response = tokenizer.decode(response_tokens, skip_special_tokens=True)
    print(f"  Decoded response: '{decoded_response}'")

print("="*60 + "\n")
print("✅ Datasets created successfully")

Train dataset size: 2000
Valid dataset size: 100
Test dataset size: 100

TOKENIZED DATASET SAMPLE INSPECTION
Dataset sample shapes:
  Input IDs shape: torch.Size([512])
  Labels shape: torch.Size([512])
  Attention mask shape: torch.Size([512])

Input IDs (first 20 tokens): [3872, 419, 3395, 6785, 476, 8225, 1939, 40, 3937, 311, 1490, 419, 264, 2421, 2849, 4134, 11, 323, 432, 594]
Labels (first 20 tokens): [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]
Attention mask (first 20 tokens): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Decoded input (first 200 chars): Is this review positive or negative?

I went to see this a few days ago, and it's hard to forget that film...for the wrong reasons. This film is supposed to be funny, it's not, not a single laugh in t...

Labels analysis:
  Prompt tokens (label=-100): 215
  Response tokens (label=actual): 2
  Padding tokens (label=-100): 295
  Decoded res

---
## Section 4: Evaluation Functions

Define functions for:
- Model inference (sentiment prediction)
- Comprehensive evaluation with metrics
- Confusion matrix generation

In [14]:
"""
Prediction function for sentiment inference

This function:
1. Takes a prompt as input
2. Generates model predictions (greedy decoding)
3. Extracts sentiment from generated text
"""
def predict_sentiment(model, tokenizer, prompt: str, max_new_tokens: int = 10) -> str:
    """
    Generate sentiment prediction for a given prompt.

    Args:
        model: Trained model
        tokenizer: Tokenizer
        prompt: Input prompt
        max_new_tokens: Maximum tokens to generate

    Returns:
        Predicted sentiment ('positive' or 'negative')
    """
    model.eval() #  Set model to eval mode so it does not update gradients
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH)
    # Remove token_type_ids if exists
    if 'token_type_ids' in inputs:
        inputs.pop('token_type_ids')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # Generate text from the model, without computing gradients
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode only the generated part (ignore the prompt)
    # Turn the token IDs back to human-readable text
    # (2 point)
    ### YOUR CODE HERE
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    ### END OF YOUR CODE

    # Extract sentiment
    # (2 point)
    ### YOUR CODE HERE
    if 'positive' in generated_text:
        return 'positive'
    elif 'negative' in generated_text:
        return 'negative'
    else:
        # Default to negative
        return 'negative'
    ### END OF YOUR CODE

print("✅ predict_sentiment function defined")

✅ predict_sentiment function defined


In [15]:
"""
Comprehensive evaluation function

This function:
1. Evaluates model on entire test set
2. Calculates multiple metrics (accuracy, F1, precision, recall)
3. Generates confusion matrix
4. Saves failed cases for analysis
"""
def evaluate_model(model, tokenizer, test_file: str, output_prefix: str) -> Dict:
    """
    Evaluate model on test set.

    Args:
        model: Trained model
        tokenizer: Tokenizer
        test_file: Path to test JSONL file
        output_prefix: Prefix for output files (e.g., 'base', 'full', 'lora')

    Returns:
        Dictionary with evaluation metrics
    """
    print(f"\nEvaluating {output_prefix} model...")

    test_data = load_jsonl(test_file)

    y_true = []
    y_pred = []
    failed_cases = []

    for item in tqdm(test_data, desc="Evaluating"):
        prompt = item['prompt']
        true_label = item['response']

        pred_label = predict_sentiment(model, tokenizer, prompt)

        y_true.append(true_label)
        y_pred.append(pred_label)

        # Collect failed cases
        if pred_label != true_label:
            failed_cases.append({
                "prompt": prompt[:200],
                "true_label": true_label,
                "predicted_label": pred_label
            })

    # Calculate metrics
    # ACC F1 Precision Recall (4 point)
    ### YOUR CODE HERE
    accuracy = accuracy_score(y_true, y_pred)
    macro_f1 = f1_score(y_true, y_pred, labels=['positive', 'negative'], average='macro')
    macro_precision = precision_score(y_true, y_pred, labels=['positive', 'negative'], average='macro')
    macro_recall = recall_score(y_true, y_pred, labels=['positive', 'negative'], average='macro')
    ### END OF YOUR CODE

    metrics = {
        "accuracy": float(accuracy),
        "macro_f1": float(macro_f1),
        "macro_precision": float(macro_precision),
        "macro_recall": float(macro_recall),
        "total_samples": len(y_true),
        "correct_predictions": int(sum([1 for t, p in zip(y_true, y_pred) if t == p]))
    }

    # Save metrics
    with open(f"logs/metrics_{output_prefix}.json", "w") as f:
        json.dump(metrics, f, indent=2)

    print(f"\n{output_prefix.upper()} Model Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Macro F1: {macro_f1:.4f}")
    print(f"Macro Precision: {macro_precision:.4f}")
    print(f"Macro Recall: {macro_recall:.4f}")

    # Save failed cases (first 5)
    if failed_cases:
        with open(f"logs/failed_cases_{output_prefix}.json", "w") as f:
            json.dump(failed_cases[:5], f, indent=2, ensure_ascii=False)
        print(f"\nFailed cases saved to logs/failed_cases_{output_prefix}.json")

    # Generate confusion matrix & show png
    # (2 point)
    ### YOUR CODE HERE
    cm = confusion_matrix(y_true, y_pred, labels=['positive', 'negative'])
    plt.figure()
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['positive','negative'],
        yticklabels=['positive','negative']
    )
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix-{output_prefix}')
    plt.savefig(f'figures/confmat_{output_prefix}.png')
    ### END OF YOUR CODE
    print(f"Confusion matrix saved to figures/confmat_{output_prefix}.png")

    return metrics
print("✅ evaluate_model function defined")

✅ evaluate_model function defined



## Section 5: Base Model Testing (Zero-Shot)

Evaluate the pre-trained Qwen model without any fine-tuning to establish a baseline performance.

In [16]:
"""
Load base model for zero-shot evaluation

This establishes our baseline - how well does the model perform
on sentiment analysis without any task-specific training?
"""
print("\n" + "="*60)
print("Base Model Testing (No Fine-tuning)")
print("="*60 + "\n")

free_gpu("before base model")

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else None,
    trust_remote_code=True,
    device_map="auto"
)

base_model.resize_token_embeddings(len(tokenizer))
base_model.config.pad_token_id = tokenizer.pad_token_id

print("✅ Base model loaded successfully")


Base Model Testing (No Fine-tuning)

[free_gpu][before base model] GPU0: alloc=0.0MB, reserved=0.0MB


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

✅ Base model loaded successfully


In [17]:
"""
Evaluate base model on test set
"""
# using before define evaluate_model function
# your code here
metrics_base = evaluate_model(base_model, tokenizer, test_file="data/test.jsonl", output_prefix="base")

# Clean up to free memory for next stage
del base_model
free_gpu("after base model")


Evaluating base model...


Evaluating: 100%|██████████| 100/100 [01:16<00:00,  1.31it/s]
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



BASE Model Metrics:
Accuracy: 0.4000
Macro F1: 0.2857
Macro Precision: 0.5000
Macro Recall: 0.2000

Failed cases saved to logs/failed_cases_base.json
Confusion matrix saved to figures/confmat_base.png
[free_gpu][after base model] GPU0: alloc=8.1MB, reserved=1136.0MB


---
## Section 6: Full-Parameter SFT [code] (10 point)

Fine-tune ALL model parameters using supervised fine-tuning (SFT).

**Approach**: Updates all 600M parameters of the Qwen model

In [18]:
"""
Load model for full-parameter fine-tuning
"""
print("\n" + "="*60)
print("Full-Parameter SFT")
print("="*60 + "\n")

free_gpu("before full SFT")

model_full = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else None,
    trust_remote_code=True,
    device_map="auto"
)

model_full.resize_token_embeddings(len(tokenizer))
model_full.config.pad_token_id = tokenizer.pad_token_id
model_full.config.use_cache = False

if hasattr(model_full, "gradient_checkpointing_enable"):
    model_full.gradient_checkpointing_enable()
    print("✅ Gradient checkpointing enabled (saves memory)")

# Count parameters
# (2 point)
### YOUR CODE HERE
total_params=sum(p.numel() for p in model_full.parameters())
trainable_params=sum(p.numel() for p in model_full.parameters()if p.requires_grad)
### END OF YOUR CODE

print(f"\nModel loaded:")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")


Full-Parameter SFT

[free_gpu][before full SFT] GPU0: alloc=8.1MB, reserved=1136.0MB
✅ Gradient checkpointing enabled (saves memory)

Model loaded:
Total parameters: 595,776,512
Trainable parameters: 595,776,512 (100.00%)


In [19]:
"""
Configure training arguments and initialize trainer

Key settings:
- bf16: Use bfloat16 for faster training and lower memory
- gradient_accumulation_steps: Simulate larger batch size
- save_strategy: Save checkpoint after each epoch
"""

# Configure the training arguments for full-parameter SFT (6 point)
training_args_full = TrainingArguments(
    ### YOUR CODE HERE
    output_dir="checkpoints/full_sft",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    bf16=True,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    logging_dir="logs/full_sft",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    report_to="none",
    seed=RANDOM_SEED
    ### END OF YOUR CODE
)

print("✅ Training arguments configured for full SFT")

trainer_full = Trainer(
    model=model_full,
    args=training_args_full,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer
)

print("✅ Trainer initialized for full SFT")

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Training arguments configured for full SFT
✅ Trainer initialized for full SFT


In [20]:
"""
Train the full-parameter model

This will take approximately 2-3 minutes on a 15GB GPU.
"""
print("\n🚀 Starting Full-Parameter SFT Training...\n")
import time
start_time = time.time()
train_result = trainer_full.train()
training_time_full = time.time() - start_time

print(f"\n✅ Training completed in {training_time_full:.2f} seconds ({training_time_full/60:.2f} minutes)")

# Save model
trainer_full.save_model("checkpoints/full_sft/final_model")
tokenizer.save_pretrained("checkpoints/full_sft/final_model")
print("✅ Model saved to checkpoints/full_sft/final_model")

# Save training metrics
train_metrics_full = {
    "training_time_seconds": training_time_full,
    "training_time_minutes": training_time_full / 60,
    "train_loss": train_result.training_loss,
    "total_params": total_params,
    "trainable_params": trainable_params,
    "trainable_params_percentage": 100 * trainable_params / total_params
}

with open("logs/full_sft_train_metrics.json", "w") as f:
    json.dump(train_metrics_full, f, indent=2)

print("✅ Training metrics saved to logs/full_sft_train_metrics.json")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.



🚀 Starting Full-Parameter SFT Training...



Epoch,Training Loss,Validation Loss
1,0.000000,0.000008



✅ Training completed in 2288.21 seconds (38.14 minutes)
✅ Model saved to checkpoints/full_sft/final_model
✅ Training metrics saved to logs/full_sft_train_metrics.json


In [21]:
"""
Evaluate full SFT model on test set
"""


metrics_full = evaluate_model(model_full, tokenizer, "data/test.jsonl", "full")


# Clean up to free memory for LoRA training
del model_full, trainer_full
free_gpu("after full SFT")


Evaluating full model...


Evaluating: 100%|██████████| 100/100 [00:20<00:00,  4.96it/s]
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", l


FULL Model Metrics:
Accuracy: 1.0000
Macro F1: 0.5000
Macro Precision: 0.5000
Macro Recall: 0.5000
Confusion matrix saved to figures/confmat_full.png
[free_gpu][after full SFT] GPU0: alloc=3426.9MB, reserved=5526.0MB


---
## Section 7: LoRA-SFT (Parameter-Efficient Fine-Tuning)

Apply LoRA (Low-Rank Adaptation) for parameter-efficient fine-tuning.

**Approach**: Only updates low-rank adapter matrices (~0.5% of parameters)
**Training time**: ~1-2 minutes (faster than full SFT)
**Memory usage**: ~8-10 GB (lower than full SFT)
**Performance**: Often comparable to full fine-tuning

In [22]:
"""
Load model and apply LoRA configuration

LoRA parameters:
- r=8: Rank of low-rank matrices (controls adapter size)
- lora_alpha=16: Scaling factor for LoRA weights
- target_modules: Which attention layers to adapt (q, k, v, o projections)
- lora_dropout=0.05: Dropout for regularization
"""
print("\n" + "="*60)
print("LoRA-SFT (Parameter-Efficient Fine-Tuning)")
print("="*60 + "\n")

free_gpu("before LoRA SFT")

model_lora = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

model_lora.resize_token_embeddings(len(tokenizer))
model_lora.config.pad_token_id = tokenizer.pad_token_id
model_lora.config.use_cache = False
if hasattr(model_lora, "gradient_checkpointing_enable"):
    model_lora.gradient_checkpointing_enable()
    print("Gradient checkpointing enabled for LoRA")
print("✅ Base model loaded for LoRA")

# Apply LoRA
# r=8, lora_alpha=16, target_modules=["q_proj", "v_proj", "k_proj", "o_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
# (4 point)
lora_config = LoraConfig(
    ### YOUR CODE HERE
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
    ### END OF YOUR CODE
)

model_lora = get_peft_model(model_lora, lora_config)

# Print trainable parameters
model_lora.print_trainable_parameters()

# Count parameters
total_params_lora = sum(p.numel() for p in model_lora.parameters())
trainable_params_lora = sum(p.numel() for p in model_lora.parameters() if p.requires_grad)

print(f"\nLoRA model:")
print(f"Total parameters: {total_params_lora:,}")
print(f"Trainable parameters: {trainable_params_lora:,} ({100 * trainable_params_lora / total_params_lora:.4f}%)")
print(f"\n📊 Parameter reduction: {100 * (1 - trainable_params_lora / trainable_params):.2f}%")


LoRA-SFT (Parameter-Efficient Fine-Tuning)

[free_gpu][before LoRA SFT] GPU0: alloc=3426.9MB, reserved=5526.0MB
Gradient checkpointing enabled for LoRA
✅ Base model loaded for LoRA
trainable params: 2,293,760 || all params: 598,070,272 || trainable%: 0.3835

LoRA model:
Total parameters: 598,070,272
Trainable parameters: 2,293,760 (0.3835%)

📊 Parameter reduction: 99.61%


#### Q7.1: Please examine the model architecture of Qwen-3-0.6B (https://huggingface.co/Qwen/Qwen3-0.6B) and mathematically derive the number of trainable parameters under the current configuration [write] (10 points)


**Base model architecture specifications (https://huggingface.co/Qwen/Qwen3-0.6B/blob/main/config.json):**

* hidden_size = 1024
* head_dim = 128
* num_attention_heads = 16
* num_key_value_heads = 8
* num_hidden_layers = 28

**LoRA params per layer**

Rank = 8

$d_Q$ = num_attention_heads × head_dim = 16×128 = 2048

$d_{KV}$ = num_key_value_heads × head_dim = 8×128 = 1024

Params for $W_Q$ = 8×(2048+1024)= 24576

Params for $W_K$ = Params for $W_V$ = 8×(1024+1024)= 16384

Params for $W_K$ = 8×(1024+2048}= 24576

Sum per layer = 24576 + 16384 + 16384 + 24576 = 81920

**Total trainable params across all 28 layers**

= 81920×28

= 2293760

In [23]:
"""
Configure training arguments and initialize trainer for LoRA
"""
training_args_lora = TrainingArguments(
    output_dir="checkpoints/lora_sft",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    bf16=True,
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    logging_dir="logs/lora_sft",
    save_total_limit=1,
    load_best_model_at_end=False,
    remove_unused_columns=False,
    report_to="none",
    seed=RANDOM_SEED,
    dataloader_num_workers=2,
    dataloader_pin_memory=False,
    max_grad_norm=1.0,
    optim="adamw_torch",
)

print("✅ Training arguments configured for LoRA SFT")

trainer_lora = Trainer(
    model=model_lora,
    args=training_args_lora,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    processing_class=tokenizer
)

print("✅ Trainer initialized for LoRA SFT")

The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Training arguments configured for LoRA SFT
✅ Trainer initialized for LoRA SFT


In [24]:
"""
Train the LoRA model

This should be faster than full SFT due to fewer trainable parameters.
"""
print("\n🚀 Starting LoRA-SFT Training...\n")

start_time = time.time()
train_result_lora = trainer_lora.train()
training_time_lora = time.time() - start_time

print(f"\n✅ Training completed in {training_time_lora:.2f} seconds ({training_time_lora/60:.2f} minutes)")

# Save LoRA adapter
model_lora.save_pretrained("checkpoints/lora_sft/final_adapter")
tokenizer.save_pretrained("checkpoints/lora_sft/final_adapter")
print("✅ LoRA adapter saved to checkpoints/lora_sft/final_adapter")

# Save training metrics
train_metrics_lora = {
    "training_time_seconds": training_time_lora,
    "training_time_minutes": training_time_lora / 60,
    "train_loss": train_result_lora.training_loss,
    "total_params": total_params_lora,
    "trainable_params": trainable_params_lora,
    "trainable_params_percentage": 100 * trainable_params_lora / total_params_lora
}

with open("logs/lora_sft_train_metrics.json", "w") as f:
    json.dump(train_metrics_lora, f, indent=2)

print("✅ Training metrics saved to logs/lora_sft_train_metrics.json")

# Print speedup
speedup = training_time_full / training_time_lora
print(f"\n⚡ LoRA training speedup: {speedup:.2f}x faster than full SFT")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.



🚀 Starting LoRA-SFT Training...



Epoch,Training Loss,Validation Loss
1,4.137000,3.326318


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(



✅ Training completed in 1916.73 seconds (31.95 minutes)


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


✅ LoRA adapter saved to checkpoints/lora_sft/final_adapter
✅ Training metrics saved to logs/lora_sft_train_metrics.json

⚡ LoRA training speedup: 1.19x faster than full SFT


In [25]:
"""
Merge LoRA weights with base model and evaluate

Merging is optional but makes deployment easier (single model file).
"""
print("\nMerging LoRA weights with base model...")
# merge (2 point)
### YOUR CODE HERE
model_lora_merged = model_lora.merge_and_unload()
### END OF YOUR CODE
model_lora_merged.save_pretrained("checkpoints/lora_sft/merged_model")
tokenizer.save_pretrained("checkpoints/lora_sft/merged_model")
print("✅ Merged model saved to checkpoints/lora_sft/merged_model")

# Evaluate LoRA SFT model
metrics_lora = evaluate_model(model_lora_merged, tokenizer, "data/test.jsonl", "lora")

# Clean up
del model_lora, model_lora_merged, trainer_lora
free_gpu("after LoRA SFT")


Merging LoRA weights with base model...
✅ Merged model saved to checkpoints/lora_sft/merged_model

Evaluating lora model...


Evaluating: 100%|██████████| 100/100 [00:55<00:00,  1.81it/s]
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



LORA Model Metrics:
Accuracy: 0.9800
Macro F1: 0.4949
Macro Precision: 0.5000
Macro Recall: 0.4900

Failed cases saved to logs/failed_cases_lora.json
Confusion matrix saved to figures/confmat_lora.png
[free_gpu][after LoRA SFT] GPU0: alloc=3426.9MB, reserved=5526.0MB



## Section 8: Performance Comparison and Visualization [code]

Compare all three approaches (Base, Full-SFT, LoRA-SFT) across multiple dimensions:
- Accuracy and F1 scores
- Training time
- Number of trainable parameters

In [26]:
"""
Print comparison table
"""
print("\n" + "="*60)
print("Performance Comparison")
print("="*60 + "\n")

models = ['Base Model', 'Full-SFT', 'LoRA-SFT']
accuracies = [
    metrics_base['accuracy'],
    metrics_full['accuracy'],
    metrics_lora['accuracy']
]
f1_scores = [
    metrics_base['macro_f1'],
    metrics_full['macro_f1'],
    metrics_lora['macro_f1']
]

print(f"{'Model':<20} {'Accuracy':<15} {'Macro F1':<15}")
print("-" * 50)
for i, model in enumerate(models):
    print(f"{model:<20} {accuracies[i]:<15.4f} {f1_scores[i]:<15.4f}")
print("\n")


Performance Comparison

Model                Accuracy        Macro F1       
--------------------------------------------------
Base Model           0.4000          0.2857         
Full-SFT             1.0000          0.5000         
LoRA-SFT             0.9800          0.4949         




In [27]:
"""
Create comparison bar charts
"""
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

# Accuracy comparison
bars1 = ax1.bar(models, accuracies, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Accuracy', fontsize=12, fontweight='bold')
ax1.set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
ax1.set_ylim(0, 1.0)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars1:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

# F1 Score comparison
bars2 = ax2.bar(models, f1_scores, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Macro F1 Score', fontsize=12, fontweight='bold')
ax2.set_title('Model F1 Score Comparison', fontsize=14, fontweight='bold')
ax2.set_ylim(0, 1.0)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

for bar in bars2:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.4f}',
             ha='center', va='bottom', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig('figures/model_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Comparison chart saved to figures/model_comparison.png")

✅ Comparison chart saved to figures/model_comparison.png


In [28]:
"""
Create comprehensive comparison with all metrics
"""
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

metrics_names = ['Accuracy', 'Macro F1', 'Precision', 'Recall']
metrics_data = [
    [metrics_base['accuracy'], metrics_full['accuracy'], metrics_lora['accuracy']],
    [metrics_base['macro_f1'], metrics_full['macro_f1'], metrics_lora['macro_f1']],
    [metrics_base['macro_precision'], metrics_full['macro_precision'], metrics_lora['macro_precision']],
    [metrics_base['macro_recall'], metrics_full['macro_recall'], metrics_lora['macro_recall']]
]

for idx, (ax, metric_name, metric_values) in enumerate(zip(axes.flat, metrics_names, metrics_data)):
    bars = ax.bar(models, metric_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    ax.set_ylabel(metric_name, fontsize=11, fontweight='bold')
    ax.set_title(f'{metric_name} Comparison', fontsize=13, fontweight='bold')
    ax.set_ylim(0, 1.0)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_xticklabels(models, rotation=15, ha='right')

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.4f}',
                ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.suptitle('Comprehensive Model Performance Comparison', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()
plt.savefig('figures/comprehensive_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ Comprehensive comparison chart saved to figures/comprehensive_comparison.png")

/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')
/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')
/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')
/tmp/ipython-input-2825173457.py:20: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(models, rotation=15, ha='right')


✅ Comprehensive comparison chart saved to figures/comprehensive_comparison.png


In [29]:
"""
Generate comprehensive project summary
"""
summary = {
    "project_info": {
        "model": MODEL_NAME,
        "dataset": "stanfordnlp/imdb",
        "task": "Binary sentiment classification",
        "random_seed": RANDOM_SEED
    },
    "configuration": {
        "max_seq_length": MAX_SEQ_LENGTH,
        "train_batch_size": TRAIN_BATCH_SIZE,
        "eval_batch_size": EVAL_BATCH_SIZE,
        "gradient_accumulation_steps": GRADIENT_ACCUMULATION_STEPS,
        "effective_batch_size": TRAIN_BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS,
        "learning_rate": LEARNING_RATE,
        "num_epochs": NUM_EPOCHS
    },
    "data_splits": {
        "train": len(train_dataset),
        "validation": len(valid_dataset),
        "test": len(test_dataset)
    },
    "base_model": {
        "test_metrics": metrics_base
    },
    "full_sft": {
        "training": train_metrics_full,
        "test_metrics": metrics_full
    },
    "lora_sft": {
        "training": train_metrics_lora,
        "test_metrics": metrics_lora
    },
    "comparison": {
        "accuracy": {
            "base": metrics_base['accuracy'],
            "full_sft": metrics_full['accuracy'],
            "lora_sft": metrics_lora['accuracy']
        },
        "training_time_minutes": {
            "full_sft": train_metrics_full['training_time_minutes'],
            "lora_sft": train_metrics_lora['training_time_minutes']
        },
        "trainable_params_millions": {
            "full_sft": train_metrics_full['trainable_params'] / 1e6,
            "lora_sft": train_metrics_lora['trainable_params'] / 1e6
        }
    }
}

with open("logs/project_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n" + "="*60)
print("PROJECT SUMMARY")
print("="*60)
print(json.dumps(summary, indent=2))
print("\n✅ Project summary saved to logs/project_summary.json")


PROJECT SUMMARY
{
  "project_info": {
    "model": "Qwen/Qwen3-0.6B",
    "dataset": "stanfordnlp/imdb",
    "task": "Binary sentiment classification",
    "random_seed": 42
  },
  "configuration": {
    "max_seq_length": 512,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "gradient_accumulation_steps": 4,
    "effective_batch_size": 32,
    "learning_rate": 2e-05,
    "num_epochs": 1
  },
  "data_splits": {
    "train": 2000,
    "validation": 100,
    "test": 100
  },
  "base_model": {
    "test_metrics": {
      "accuracy": 0.4,
      "macro_f1": 0.2857142857142857,
      "macro_precision": 0.5,
      "macro_recall": 0.2,
      "total_samples": 100,
      "correct_predictions": 40
    }
  },
  "full_sft": {
    "training": {
      "training_time_seconds": 2288.2104122638702,
      "training_time_minutes": 38.136840204397835,
      "train_loss": 1.6306184718625791,
      "total_params": 595776512,
      "trainable_params": 595776512,
      "trainable_params_percentage": 10

In [30]:
"""
Print final comparison table
"""
print("\n" + "="*80)
print("FINAL COMPARISON TABLE")
print("="*80)
print(f"{'Metric':<30} {'Base Model':<20} {'Full-SFT':<20} {'LoRA-SFT':<20}")
print("-"*80)
print(f"{'Test Accuracy':<30} {metrics_base['accuracy']:>19.4f} {metrics_full['accuracy']:>19.4f} {metrics_lora['accuracy']:>19.4f}")
print(f"{'Test Macro F1':<30} {metrics_base['macro_f1']:>19.4f} {metrics_full['macro_f1']:>19.4f} {metrics_lora['macro_f1']:>19.4f}")
print(f"{'Test Precision':<30} {metrics_base['macro_precision']:>19.4f} {metrics_full['macro_precision']:>19.4f} {metrics_lora['macro_precision']:>19.4f}")
print(f"{'Test Recall':<30} {metrics_base['macro_recall']:>19.4f} {metrics_full['macro_recall']:>19.4f} {metrics_lora['macro_recall']:>19.4f}")
print("-"*80)
print(f"{'Training Time (min)':<30} {'N/A':>19} {train_metrics_full['training_time_minutes']:>19.2f} {train_metrics_lora['training_time_minutes']:>19.2f}")
print(f"{'Trainable Params (M)':<30} {'N/A':>19} {train_metrics_full['trainable_params']/1e6:>19.2f} {train_metrics_lora['trainable_params']/1e6:>19.2f}")
print(f"{'Trainable Params %':<30} {'N/A':>19} {train_metrics_full['trainable_params_percentage']:>19.2f} {train_metrics_lora['trainable_params_percentage']:>19.2f}")
print("="*80)

print("\n✅ All tasks completed successfully!")
print("\n📁 Generated files:")
print("  - Data: data/train.jsonl, data/valid.jsonl, data/test.jsonl")
print("  - Models: checkpoints/full_sft/, checkpoints/lora_sft/")
print("  - Metrics: logs/metrics_*.json")
print("  - Figures: figures/*.png")
print("  - Summary: logs/project_summary.json")


FINAL COMPARISON TABLE
Metric                         Base Model           Full-SFT             LoRA-SFT            
--------------------------------------------------------------------------------
Test Accuracy                               0.4000              1.0000              0.9800
Test Macro F1                               0.2857              0.5000              0.4949
Test Precision                              0.5000              0.5000              0.5000
Test Recall                                 0.2000              0.5000              0.4900
--------------------------------------------------------------------------------
Training Time (min)                            N/A               38.14               31.95
Trainable Params (M)                           N/A              595.78                2.29
Trainable Params %                             N/A              100.00                0.38

✅ All tasks completed successfully!

📁 Generated files:
  - Data: data/train.jsonl


#### Q8.1: Why is LoRA-SFT training time shorter than full-parameter SFT? Provide a detailed explanation considering: Number of parameters being updated, Gradient computation complexity, Memory bandwidth requirements[written] (5 points)




Lora-SFT has a shorter training time because it updates only about 0.38% of the total number of trainable parameters (about 2.29 million). It works by introducing small low-rank adapter matrices into selected attention layers, and only these adapters are trained while the original model weights remain frozen, so backpropogation is performed only through these low-rank matrices.

In contrast, full-parameter SFT computes gradients, stores activations, and maintains optimizer states for all 595.78 million trainable parameters, which leads to high computational cost and large memory required.

Therefore, the gradient computation complexity, which is quadratic O($d^2$) for full-parameter SFT, is reduced to O($r·d$) , where r=8 and d=1024.

Memory bandwidth requirements are also reduced because the optimizer does not need to read, write or update the large frozen weight matrices.

### Other Question

#### Q9: For example, if we switch to a different prompting format, will accuracy drop significantly when the train and test prompts are not completely consistent? Conduct an experiment to demonstrate this: (10 points)
*   Design an alternative prompt format
*   Test the trained model with the new format
*   Report and analyze the results

In [37]:
torch.manual_seed(RANDOM_SEED)

def format_alt_prompt(text):
    return f"Assess whether this review is positive or negative: {text}"

model_lora_merged = AutoModelForCausalLM.from_pretrained("checkpoints/lora_sft/merged_model")
model_lora_merged.eval()  # set to evaluation mode
tokenizer = AutoTokenizer.from_pretrained("checkpoints/lora_sft/merged_model", trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def chat_with_model(prompt, max_new_tokens=100, temperature=0.6):
    device = next(model_lora_merged.parameters()).device
    inputs = tokenizer(
        prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH
    ).to(device)

    with torch.inference_mode():
        outputs = model_lora_merged.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True
        )

    seq = outputs.sequences[0]
    gen_only = seq[inputs["input_ids"].shape[1]:]
    return tokenizer.decode(gen_only, skip_special_tokens=True)

y_true = []
y_pred = []

for item in test_subset:
    prompt = format_alt_prompt(item['text'])
    true_label = item['label']  # 'positive' or 'negative'
    pred_text = chat_with_model(prompt, max_new_tokens=10).lower().strip()

    # Map output to 'positive' or 'negative'
    if 'positive' in pred_text:
        pred_label = 'positive'
    elif 'negative' in pred_text:
        pred_label = 'negative'
    else:
        pred_label = 'negative'  # default fallback

    y_true.append(true_label)
    y_pred.append(pred_label)

accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')
macro_precision = precision_score(y_true, y_pred, average='macro')
macro_recall = recall_score(y_true, y_pred, average='macro')

print("\n--- Evaluation on Alternative Prompt Format ---")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1: {macro_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")


--- Evaluation on Alternative Prompt Format ---
Accuracy: 0.5500
Macro F1: 0.4479
Macro Precision: 0.6920
Macro Recall: 0.5500


As seen, the test accuracy for LoRA-SFT drops significantly from 98% to 55% when the prompt is changed, showing that the model struggles to generalise to different instruction formats.

The macro F1 also drops from 0.4949 to 0.4478, showing that the balance between precision and recall is poorer. The model is more conservative in predicting positive cases, avoiding false positives but missing many true positives.



#### Q10: How can we increase robustness in this scenario, given that users’ queries are highly diverse? (5 points)


We can train the model on more instruction-response pairs. Currently, it is only trained on one fixed prompt format. We can add more, such as "Classify this review: {text}", or Assess if this review {text} is positive or negative."

For these additional instructions, it would be good to have different action verbs like "Classify","Predict","Assess", and different styles of writing like casual, question, or command style. This increased diversity of prompts can help the model generalize to unseen instructions better, increasing the robustness against highly diverse user queries.

#### Q11: Please propose five different test cases that will cause the SFT-trained model to classify incorrectly, and verify them. (5 points)

**Five test cases:**

1. Sarcasm
2. Mixed sentiment
3. Ambiguous sentiment
4. Out-of-vocabulary words (e.g. latest slang)
5. Figurative language

In [49]:
torch.manual_seed(RANDOM_SEED)

model_sft = AutoModelForCausalLM.from_pretrained("checkpoints/full_sft/final_model")
model_sft.eval()
tokenizer = AutoTokenizer.from_pretrained("checkpoints/full_sft/final_model", trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def chat_with_model(prompt, max_new_tokens=100, temperature=0.6):
    device = next(model_sft.parameters()).device
    inputs = tokenizer(
        prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LENGTH
    ).to(device)

    with torch.inference_mode():
        outputs = model_sft.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True
        )
    seq = outputs.sequences[0]
    gen_only = seq[inputs["input_ids"].shape[1]:]
    return tokenizer.decode(gen_only, skip_special_tokens=True)

test_cases = [
    ("This movie is so good, it made me fall asleep.", "negative"),  # sarcasm
    ("The movie was good, though the ending should have been better.", "negative"),   # mixed sentiment
    ("I cried a lot.", "positive"),  # ambiguous
    ("This movie is so goated!", "positive"),    # slang / OOV
    ("The film was a rollercoaster of emotions.", "positive")  # figurative language
]

y_true = []
y_pred = []

for text, true_label in test_cases:
    prompt = format_alt_prompt(text)
    pred_text = chat_with_model(prompt, max_new_tokens=10).lower().strip()

    # Map generated text to 'positive' or 'negative'
    if 'positive' in pred_text:
        pred_label = 'positive'
    elif 'negative' in pred_text:
        pred_label = 'negative'
    else:
        pred_label = 'negative'  # fallback

    y_true.append(true_label)
    y_pred.append(pred_label)

    # Print each case
    print(f"Review: {text}")
    print(f"True label: {true_label}, Predicted: {pred_label}")
    print(f"{'✅ Correct' if pred_label == true_label else '❌ Incorrect'}\n")

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')
macro_precision = precision_score(y_true, y_pred, average='macro')
macro_recall = recall_score(y_true, y_pred, average='macro')

print("--- Evaluation on Five Challenging Test Cases ---")
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1: {macro_f1:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")

Review: This movie is so good, it made me fall asleep.
True label: negative, Predicted: positive
❌ Incorrect

Review: The movie was good, though the ending should have been better.
True label: negative, Predicted: positive
❌ Incorrect

Review: I cried a lot.
True label: positive, Predicted: negative
❌ Incorrect

Review: This movie is so goated!
True label: positive, Predicted: negative
❌ Incorrect

Review: The film was a rollercoaster of emotions.
True label: positive, Predicted: negative
❌ Incorrect

--- Evaluation on Five Challenging Test Cases ---
Accuracy: 0.0000
Macro F1: 0.0000
Macro Precision: 0.0000
Macro Recall: 0.0000
